# 🧠 Fine-tuning Vision-Language Model pour Surveillance

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/elfried-kinzoun/intelligent-surveillance-system/blob/main/notebooks/1_fine_tuning_vlm.ipynb)

**Objectif**: Fine-tuner un modèle Vision-Language (LLaVA/BLIP) spécialement pour la surveillance en grande distribution avec capacités de tool-calling.

## 🎯 Ce que vous allez apprendre :
- 📊 **Préparation de dataset** spécialisé surveillance
- 🔧 **Fine-tuning LoRA** pour optimiser les ressources
- 🛠️ **Tool-calling integration** pour orchestration d'outils
- 📈 **Évaluation qualitative** des performances
- 🚀 **Déploiement** du modèle fine-tuné

## ⚙️ Configuration Recommandée :
- **GPU**: T4 (Colab gratuit) ou A100 (Colab Pro)
- **RAM**: 12+ GB
- **Temps**: 2-4 heures selon GPU

## 🚀 Configuration Initiale

In [ ]:
# Installation des dépendances spécialisées pour fine-tuning
!pip install -q torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118
!pip install -q transformers[torch]==4.36.0
!pip install -q peft==0.7.0  # Parameter Efficient Fine-Tuning
!pip install -q bitsandbytes==0.41.3  # Quantization
!pip install -q accelerate==0.25.0
!pip install -q datasets==2.15.0
!pip install -q wandb  # Weights & Biases pour tracking
!pip install -q deepspeed  # Optimisation mémoire
!pip install -q flash-attn --no-build-isolation  # Attention optimisée

# Installation du projet
!git clone -q https://github.com/elfried-kinzoun/intelligent-surveillance-system.git
%cd intelligent-surveillance-system
!pip install -q -e .

print("✅ Installation terminée!")

In [ ]:
import torch
import transformers
from transformers import AutoTokenizer, AutoProcessor
import os
from pathlib import Path

# Vérification GPU et configuration
print(f"🔥 PyTorch: {torch.__version__}")
print(f"🤗 Transformers: {transformers.__version__}")
print(f"🎮 CUDA disponible: {torch.cuda.is_available()}")

if torch.cuda.is_available():
    gpu_name = torch.cuda.get_device_name(0)
    gpu_memory = torch.cuda.get_device_properties(0).total_memory / 1e9
    print(f"🎯 GPU: {gpu_name}")
    print(f"💾 Mémoire GPU: {gpu_memory:.1f} GB")
    
    # Recommandations selon GPU
    if "T4" in gpu_name:
        print("💡 GPU T4 détecté - Utilisation LoRA + quantization 4-bit recommandée")
        RECOMMENDED_CONFIG = {
            "load_in_4bit": True,
            "use_lora": True,
            "lora_rank": 16,
            "batch_size": 1,
            "gradient_checkpointing": True
        }
    elif "A100" in gpu_name or "V100" in gpu_name:
        print("🚀 GPU haute performance - Configuration optimale possible")
        RECOMMENDED_CONFIG = {
            "load_in_4bit": False,
            "use_lora": True,
            "lora_rank": 64,
            "batch_size": 4,
            "gradient_checkpointing": False
        }
else:
    print("⚠️ Pas de GPU - Fine-tuning sur CPU non recommandé")
    RECOMMENDED_CONFIG = {
        "load_in_4bit": False,
        "use_lora": True,
        "lora_rank": 8,
        "batch_size": 1,
        "gradient_checkpointing": True
    }

print(f"\n⚙️ Configuration recommandée: {RECOMMENDED_CONFIG}")

In [ ]:
# Configuration Weights & Biases (optionnel)
import wandb

# Décommenter et configurer si vous voulez tracker avec W&B
# wandb.login()  # Vous devrez entrer votre clé API

# Configuration du projet
PROJECT_NAME = "surveillance-vlm-finetuning"
RUN_NAME = f"llava-surveillance-{torch.cuda.get_device_name(0).replace(' ', '_') if torch.cuda.is_available() else 'cpu'}"

# Initialisation W&B (en mode offline si pas de login)
try:
    wandb.init(
        project=PROJECT_NAME,
        name=RUN_NAME,
        config=RECOMMENDED_CONFIG,
        mode="online"  # Changez en "offline" si pas de compte W&B
    )
    print("✅ W&B configuré")
except:
    print("⚠️ W&B en mode offline - tracking local uniquement")
    os.environ["WANDB_MODE"] = "offline"

## 📊 Préparation du Dataset de Surveillance

In [ ]:
import json
import random
from datasets import Dataset
from PIL import Image
import requests
from io import BytesIO

# Création d'un dataset de surveillance synthétique pour démonstration
def create_surveillance_dataset():
    """
    Crée un dataset d'entraînement pour la surveillance.
    En production, vous utiliseriez vos vraies données annotées.
    """
    
    # Templates de conversations pour différents scénarios
    conversation_templates = {
        "normal_shopping": [
            {
                "human": "Analysez cette scène de surveillance et déterminez le niveau de suspicion.",
                "assistant": "Je vois une personne qui fait ses courses normalement. Elle examine des produits, les place dans son panier de manière visible. Son comportement est cohérent avec un shopping classique. Niveau de suspicion: FAIBLE. Aucune action particulière requise. Outils recommandés: surveillance passive avec object_detector."
            },
            {
                "human": "Que voyez-vous dans cette image de magasin ? Y a-t-il quelque chose d'inhabituel ?",
                "assistant": "Cette scène montre une activité commerciale normale. Les clients se déplacent calmement, consultent les produits et interagissent naturellement avec l'environnement. Aucun comportement suspect détecté. Recommandation: continuer la surveillance de routine."
            }
        ],
        
        "suspicious_movement": [
            {
                "human": "Analysez le comportement des personnes dans cette zone de surveillance.",
                "assistant": "J'observe des mouvements erratiques qui méritent attention. Une personne semble éviter les caméras et fait des allers-retours sans acheter. Niveau de suspicion: MOYEN. Actions recommandées: surveillance renforcée, utilisation du tracker pour suivre les déplacements, activation du behavior_analyzer pour analyser les patterns."
            },
            {
                "human": "Cette personne vous semble-t-elle suspecte ? Utilisez vos outils d'analyse.",
                "assistant": "Le comportement observé présente plusieurs indicateurs d'alerte. Je recommande d'utiliser immédiatement le tracker pour suivre cette personne et le behavior_analyzer pour évaluer ses patterns de mouvement. Surveillance discrète mais active requise."
            }
        ],
        
        "potential_theft": [
            {
                "human": "Situation d'urgence - analysez immédiatement cette scène.",
                "assistant": "ALERTE: Comportement hautement suspect détecté. Je vois une personne dissimuler des objets. Niveau de suspicion: CRITIQUE. Actions immédiates: activation de tous les outils (object_detector pour identifier les objets, tracker pour suivi continu, behavior_analyzer pour confirmation, false_positive_filter pour validation). Alerte sécurité recommandée."
            }
        ],
        
        "tool_calling_example": [
            {
                "human": "Analysez cette scène et utilisez les outils appropriés pour une évaluation complète.",
                "assistant": "Je vais analyser cette scène systématiquement. Permettez-moi d'utiliser les outils disponibles.\n\n[TOOL_CALL: object_detector]\n{\"confidence_threshold\": 0.3, \"focus_classes\": [\"person\", \"handbag\", \"backpack\"]}\n\n[TOOL_CALL: behavior_analyzer]\n{\"analysis_window\": 30, \"focus_behaviors\": [\"movement_pattern\", \"interaction_objects\"]}\n\nBasé sur les résultats des outils, je détecte 2 personnes avec des sacs. Les patterns de mouvement semblent normaux. Niveau de suspicion: FAIBLE à MOYEN selon la durée d'observation."
            }
        ]
    }
    
    # Génération du dataset
    dataset_entries = []
    
    # URLs d'images de démonstration (remplacez par vos vraies images)
    demo_images = [
        "https://images.unsplash.com/photo-1556742049-0cfed4f6a45d?w=640",  # Store
        "https://images.unsplash.com/photo-1441986300917-64674bd600d8?w=640",  # Shopping
        "https://images.unsplash.com/photo-1472851294608-062f824d29cc?w=640",  # Retail
        "https://images.unsplash.com/photo-1560472354-b33ff0c44a43?w=640",  # Supermarket
        "https://images.unsplash.com/photo-1578662996442-48f60103fc96?w=640",  # Store aisle
    ]
    
    # Génération des entrées du dataset
    for scenario_type, conversations in conversation_templates.items():
        for i, conv in enumerate(conversations):
            for img_idx, img_url in enumerate(demo_images):
                entry = {
                    "id": f"{scenario_type}_{i}_{img_idx}",
                    "image_url": img_url,
                    "conversations": [
                        {"from": "human", "value": conv["human"]},
                        {"from": "gpt", "value": conv["assistant"]}
                    ],
                    "scenario": scenario_type,
                    "metadata": {
                        "domain": "surveillance",
                        "task": "security_analysis",
                        "has_tool_calling": "tool_calling" in scenario_type
                    }
                }
                dataset_entries.append(entry)
    
    return dataset_entries

# Création du dataset
print("📊 Création du dataset de surveillance...")
dataset_entries = create_surveillance_dataset()
print(f"✅ Dataset créé: {len(dataset_entries)} exemples")

# Affichage d'un exemple
print("\n📝 Exemple de conversation:")
example = dataset_entries[0]
print(f"Scénario: {example['scenario']}")
print(f"Human: {example['conversations'][0]['value']}")
print(f"Assistant: {example['conversations'][1]['value'][:200]}...")

In [ ]:
from datasets import Dataset, DatasetDict
import random

def download_and_prepare_images(dataset_entries, max_images=20):
    """
    Télécharge les images et prépare le dataset final.
    En production, vous travailleriez avec des images locales.
    """
    
    prepared_data = []
    downloaded_images = {}
    
    print(f"📥 Téléchargement de {min(max_images, len(dataset_entries))} images...")
    
    for i, entry in enumerate(dataset_entries[:max_images]):
        try:
            # Download image si pas déjà fait
            img_url = entry["image_url"]
            if img_url not in downloaded_images:
                response = requests.get(img_url, timeout=10)
                if response.status_code == 200:
                    image = Image.open(BytesIO(response.content)).convert("RGB")
                    # Resize pour optimiser
                    image = image.resize((512, 512), Image.Resampling.LANCZOS)
                    downloaded_images[img_url] = image
                else:
                    print(f"⚠️ Erreur téléchargement image {img_url}")
                    continue
            
            # Préparation de l'entrée
            prepared_entry = {
                "id": entry["id"],
                "image": downloaded_images[img_url],
                "conversations": entry["conversations"],
                "scenario": entry["scenario"]
            }
            prepared_data.append(prepared_entry)
            
            if (i + 1) % 5 == 0:
                print(f"  📷 {i + 1}/{min(max_images, len(dataset_entries))} images téléchargées")
                
        except Exception as e:
            print(f"❌ Erreur pour {entry['id']}: {e}")
            continue
    
    return prepared_data

# Préparation des données
prepared_data = download_and_prepare_images(dataset_entries, max_images=20)
print(f"✅ {len(prepared_data)} exemples préparés")

# Division train/validation
random.shuffle(prepared_data)
split_idx = int(0.8 * len(prepared_data))
train_data = prepared_data[:split_idx]
val_data = prepared_data[split_idx:]

print(f"📊 Division dataset:")
print(f"  🏋️ Train: {len(train_data)} exemples")
print(f"  📋 Validation: {len(val_data)} exemples")

# Création des datasets Hugging Face
train_dataset = Dataset.from_list(train_data)
val_dataset = Dataset.from_list(val_data)

dataset_dict = DatasetDict({
    "train": train_dataset,
    "validation": val_dataset
})

print("\n📈 Statistiques par scénario:")
scenario_counts = {}
for entry in prepared_data:
    scenario = entry["scenario"]
    scenario_counts[scenario] = scenario_counts.get(scenario, 0) + 1

for scenario, count in scenario_counts.items():
    print(f"  {scenario}: {count} exemples")

## 🧠 Configuration du Modèle VLM

In [ ]:
from transformers import (
    LlavaNextForConditionalGeneration,
    LlavaNextProcessor,
    BitsAndBytesConfig,
    TrainingArguments
)
from peft import LoraConfig, get_peft_model, TaskType
import torch

# Configuration du modèle de base
MODEL_NAME = "llava-hf/llava-v1.6-mistral-7b-hf"  # Modèle de base

print(f"🧠 Chargement du modèle: {MODEL_NAME}")
print("⏳ Cela peut prendre quelques minutes...")

# Configuration de quantization pour optimiser la mémoire
if RECOMMENDED_CONFIG["load_in_4bit"]:
    bnb_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_compute_dtype=torch.float16,
        bnb_4bit_use_double_quant=True,
    )
    print("🔧 Quantization 4-bit activée")
else:
    bnb_config = None
    print("🔧 Pas de quantization")

# Chargement du processeur
processor = LlavaNextProcessor.from_pretrained(MODEL_NAME)

# Chargement du modèle
try:
    model = LlavaNextForConditionalGeneration.from_pretrained(
        MODEL_NAME,
        quantization_config=bnb_config,
        device_map="auto" if torch.cuda.is_available() else None,
        torch_dtype=torch.float16 if torch.cuda.is_available() else torch.float32,
        attn_implementation="flash_attention_2" if torch.cuda.is_available() else "eager",
    )
    print("✅ Modèle chargé avec succès")
except Exception as e:
    print(f"⚠️ Erreur flash attention, fallback vers eager: {e}")
    model = LlavaNextForConditionalGeneration.from_pretrained(
        MODEL_NAME,
        quantization_config=bnb_config,
        device_map="auto" if torch.cuda.is_available() else None,
        torch_dtype=torch.float16 if torch.cuda.is_available() else torch.float32,
    )
    print("✅ Modèle chargé (mode standard)")

# Information sur le modèle
print(f"\n📊 Informations modèle:")
print(f"  Paramètres: ~7B")
print(f"  Device map: {model.hf_device_map if hasattr(model, 'hf_device_map') else 'auto'}")
print(f"  Dtype: {model.dtype}")
print(f"  Quantization: {'4-bit' if bnb_config else 'None'}")

In [ ]:
from peft import LoraConfig, get_peft_model, TaskType, prepare_model_for_kbit_training

# Configuration LoRA pour fine-tuning efficace
if RECOMMENDED_CONFIG["use_lora"]:
    print("🔧 Configuration LoRA...")
    
    # Préparation du modèle pour quantization si nécessaire
    if RECOMMENDED_CONFIG["load_in_4bit"]:
        model = prepare_model_for_kbit_training(model)
        print("  ✅ Modèle préparé pour quantization")
    
    # Configuration LoRA
    lora_config = LoraConfig(
        task_type=TaskType.CAUSAL_LM,  # Task type pour generation
        r=RECOMMENDED_CONFIG["lora_rank"],  # Rank de la décomposition
        lora_alpha=32,  # Scaling parameter
        lora_dropout=0.1,  # Dropout pour régularisation
        bias="none",  # Pas de bias dans LoRA
        target_modules=[
            "q_proj",
            "v_proj", 
            "k_proj",
            "o_proj",
            "gate_proj",
            "up_proj",
            "down_proj",
        ],
        inference_mode=False,
    )
    
    # Application de LoRA
    model = get_peft_model(model, lora_config)
    
    print("  ✅ LoRA configuré")
    print(f"  📊 Rank: {RECOMMENDED_CONFIG['lora_rank']}")
    print(f"  📊 Alpha: 32")
    print(f"  📊 Dropout: 0.1")
    
    # Affichage des paramètres entraînables
    trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
    total_params = sum(p.numel() for p in model.parameters())
    
    print(f"\n📈 Paramètres:")
    print(f"  Entraînables: {trainable_params:,}")
    print(f"  Total: {total_params:,}")
    print(f"  Pourcentage entraînable: {100 * trainable_params / total_params:.2f}%")
else:
    print("🔧 Fine-tuning complet (pas de LoRA)")

# Configuration du processeur pour le padding
if processor.tokenizer.pad_token is None:
    processor.tokenizer.pad_token = processor.tokenizer.eos_token
    print("✅ Pad token configuré")

## 🔄 Traitement des Données pour l'Entraînement

In [ ]:
from dataclasses import dataclass
from typing import Dict, List, Any
import torch

@dataclass
class LlavaDataCollator:
    """
    Data collator spécialisé pour LLaVA avec conversations multi-tours.
    """
    processor: Any
    
    def __call__(self, batch: List[Dict[str, Any]]) -> Dict[str, torch.Tensor]:
        # Extraction des images et conversations
        images = [item["image"] for item in batch]
        conversations = [item["conversations"] for item in batch]
        
        # Préparation des prompts pour LLaVA
        texts = []
        for conv in conversations:
            # Format de conversation pour LLaVA
            conversation_text = ""
            for turn in conv:
                if turn["from"] == "human":
                    conversation_text += f"<image>\nUSER: {turn['value']}\nASSISTANT: "
                elif turn["from"] == "gpt":
                    conversation_text += f"{turn['value']}"
            texts.append(conversation_text)
        
        # Traitement par le processeur LLaVA
        inputs = self.processor(
            text=texts,
            images=images,
            return_tensors="pt",
            padding=True,
            truncation=True,
            max_length=512
        )
        
        # Préparation des labels pour l'entraînement
        labels = inputs["input_ids"].clone()
        
        # Masquer les tokens du prompt (ne calculer la loss que sur la réponse)
        for i, text in enumerate(texts):
            # Trouver le début de la réponse de l'assistant
            assistant_start = text.find("ASSISTANT: ") + len("ASSISTANT: ")
            if assistant_start > len("ASSISTANT: "):
                # Tokeniser juste le prompt pour trouver où commencer la loss
                prompt_text = text[:assistant_start]
                prompt_tokens = self.processor.tokenizer(
                    prompt_text, 
                    add_special_tokens=False
                )["input_ids"]
                
                # Masquer les tokens du prompt
                labels[i, :len(prompt_tokens)] = -100
        
        inputs["labels"] = labels
        
        return inputs

# Création du data collator
data_collator = LlavaDataCollator(processor=processor)
print("✅ Data collator créé")

# Test du data collator
print("\n🧪 Test du data collator...")
sample_batch = [train_dataset[0], train_dataset[1] if len(train_dataset) > 1 else train_dataset[0]]
try:
    test_batch = data_collator(sample_batch)
    print(f"  ✅ Batch shape: {test_batch['input_ids'].shape}")
    print(f"  ✅ Images shape: {test_batch['pixel_values'].shape}")
    print(f"  ✅ Labels shape: {test_batch['labels'].shape}")
except Exception as e:
    print(f"  ❌ Erreur test batch: {e}")

## 🏋️ Configuration de l'Entraînement

In [ ]:
from transformers import TrainingArguments, Trainer
import math

# Configuration des paramètres d'entraînement
OUTPUT_DIR = "./surveillance-llava-finetuned"
LOGGING_DIR = "./logs"

# Calcul automatique des steps selon le dataset
num_train_samples = len(train_dataset)
batch_size = RECOMMENDED_CONFIG["batch_size"]
gradient_accumulation_steps = max(1, 4 // batch_size)  # Simuler batch size 4
effective_batch_size = batch_size * gradient_accumulation_steps

# Paramètres d'entraînement adaptés
num_epochs = 3  # Nombre d'époques
steps_per_epoch = math.ceil(num_train_samples / effective_batch_size)
total_steps = steps_per_epoch * num_epochs
warmup_steps = int(0.1 * total_steps)  # 10% de warmup
eval_steps = max(1, steps_per_epoch // 2)  # Évaluation 2x par époque
save_steps = eval_steps

print(f"📊 Configuration d'entraînement:")
print(f"  Samples d'entraînement: {num_train_samples}")
print(f"  Batch size: {batch_size}")
print(f"  Gradient accumulation: {gradient_accumulation_steps}")
print(f"  Batch size effectif: {effective_batch_size}")
print(f"  Époques: {num_epochs}")
print(f"  Steps par époque: {steps_per_epoch}")
print(f"  Total steps: {total_steps}")
print(f"  Warmup steps: {warmup_steps}")
print(f"  Eval/Save steps: {eval_steps}")

# Arguments d'entraînement
training_args = TrainingArguments(
    output_dir=OUTPUT_DIR,
    
    # Paramètres de base
    num_train_epochs=num_epochs,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    
    # Optimisation
    learning_rate=2e-4,  # Learning rate pour LoRA
    weight_decay=0.01,
    adam_beta1=0.9,
    adam_beta2=0.999,
    adam_epsilon=1e-8,
    max_grad_norm=1.0,
    
    # Scheduler
    lr_scheduler_type="cosine",
    warmup_steps=warmup_steps,
    
    # Évaluation et sauvegarde
    eval_steps=eval_steps,
    evaluation_strategy="steps",
    save_steps=save_steps,
    save_strategy="steps",
    save_total_limit=3,
    load_best_model_at_end=True,
    metric_for_best_model="eval_loss",
    greater_is_better=False,
    
    # Logging
    logging_dir=LOGGING_DIR,
    logging_steps=max(1, eval_steps // 4),
    report_to=["wandb"] if "wandb" in globals() else [],
    
    # Optimisations mémoire
    gradient_checkpointing=RECOMMENDED_CONFIG["gradient_checkpointing"],
    dataloader_pin_memory=False,
    dataloader_num_workers=2 if torch.cuda.is_available() else 0,
    
    # Précision mixte
    fp16=torch.cuda.is_available(),
    bf16=False,  # Disponible sur A100/H100
    
    # Autres
    remove_unused_columns=False,  # Important pour les données multimodales
    push_to_hub=False,
    hub_model_id=None,
)

print("\n✅ Arguments d'entraînement configurés")

In [ ]:
from transformers import Trainer
import numpy as np

# Fonction de calcul des métriques d'évaluation
def compute_metrics(eval_pred):
    """
    Calcule les métriques d'évaluation pour le fine-tuning.
    """
    predictions, labels = eval_pred
    
    # Pour les modèles génératifs, on se concentre sur la perplexité
    # Calculée automatiquement via la loss
    
    return {
        "perplexity": np.exp(np.mean(predictions)),
    }

# Classe Trainer personnalisée pour surveillance VLM
class SurveillanceVLMTrainer(Trainer):
    """
    Trainer personnalisé avec des fonctionnalités spécifiques à la surveillance.
    """
    
    def compute_loss(self, model, inputs, return_outputs=False):
        """
        Calcul de la loss avec gestion spéciale pour les modèles multimodaux.
        """
        # Forward pass
        outputs = model(**inputs)
        loss = outputs.loss
        
        # Log des métriques additionnelles
        if self.state.global_step % self.args.logging_steps == 0:
            self.log({
                "train_loss": loss.item(),
                "learning_rate": self.get_lr()[0],
                "epoch": self.state.epoch,
            })
        
        return (loss, outputs) if return_outputs else loss
    
    def evaluation_loop(self, dataloader, description, prediction_loss_only=None, ignore_keys=None, metric_key_prefix="eval"):
        """
        Loop d'évaluation personnalisé.
        """
        output = super().evaluation_loop(
            dataloader, description, prediction_loss_only, ignore_keys, metric_key_prefix
        )
        
        # Calcul de métriques supplémentaires
        if output.metrics:
            perplexity = np.exp(output.metrics.get(f"{metric_key_prefix}_loss", 0))
            output.metrics[f"{metric_key_prefix}_perplexity"] = perplexity
        
        return output

# Création du trainer
trainer = SurveillanceVLMTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    data_collator=data_collator,
    compute_metrics=None,  # Désactivé pour économiser de la mémoire
)

print("✅ Trainer configuré")
print(f"📊 Modèle sur device: {next(model.parameters()).device}")

# Vérification de la mémoire GPU si disponible
if torch.cuda.is_available():
    gpu_memory = torch.cuda.get_device_properties(0).total_memory / 1e9
    allocated_memory = torch.cuda.memory_allocated() / 1e9
    print(f"💾 Mémoire GPU: {allocated_memory:.1f}/{gpu_memory:.1f} GB utilisée")
    
    if allocated_memory / gpu_memory > 0.9:
        print("⚠️ Mémoire GPU proche de la limite - Considérez réduire batch_size")

## 🚀 Lancement de l'Entraînement

In [ ]:
import time
from datetime import datetime

print("🚀 DÉBUT DE L'ENTRAÎNEMENT")
print("=" * 50)
print(f"🕐 Heure de début: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
print(f"⏱️ Durée estimée: ~{total_steps * 30 / 60:.1f} minutes (estimation)")
print(f"📊 Configuration GPU: {torch.cuda.get_device_name(0) if torch.cuda.is_available() else 'CPU'}")
print()

# Sauvegarde des métriques initiales
start_time = time.time()
initial_memory = torch.cuda.memory_allocated() / 1e9 if torch.cuda.is_available() else 0

try:
    # Lancement de l'entraînement
    training_result = trainer.train()
    
    # Calcul des métriques finales
    end_time = time.time()
    training_duration = end_time - start_time
    final_memory = torch.cuda.memory_allocated() / 1e9 if torch.cuda.is_available() else 0
    
    print("\n" + "=" * 50)
    print("🎉 ENTRAÎNEMENT TERMINÉ AVEC SUCCÈS !")
    print("=" * 50)
    print(f"🕐 Durée totale: {training_duration/60:.1f} minutes")
    print(f"📈 Loss finale: {training_result.training_loss:.4f}")
    print(f"⚡ Steps/seconde: {training_result.global_step/training_duration:.2f}")
    print(f"💾 Mémoire GPU finale: {final_memory:.1f} GB")
    
    # Sauvegarde du modèle final
    print("\n💾 Sauvegarde du modèle...")
    trainer.save_model()
    processor.save_pretrained(OUTPUT_DIR)
    
    print(f"✅ Modèle sauvegardé dans: {OUTPUT_DIR}")
    
except Exception as e:
    print(f"\n❌ ERREUR PENDANT L'ENTRAÎNEMENT: {e}")
    print("\n💡 Solutions possibles:")
    print("  - Réduire le batch_size")
    print("  - Activer gradient_checkpointing")
    print("  - Utiliser une quantization plus agressive")
    print("  - Réduire la longueur de séquence")
    
    # Tentative de libération mémoire
    if torch.cuda.is_available():
        torch.cuda.empty_cache()
    
    raise e

## 📊 Évaluation du Modèle Fine-tuné

In [ ]:
from transformers import pipeline
import matplotlib.pyplot as plt
from PIL import Image
import requests

# Évaluation sur le set de validation
print("📊 ÉVALUATION DU MODÈLE FINE-TUNÉ")
print("=" * 40)

try:
    eval_results = trainer.evaluate()
    
    print("📈 Résultats d'évaluation:")
    for key, value in eval_results.items():
        if isinstance(value, float):
            print(f"  {key}: {value:.4f}")
        else:
            print(f"  {key}: {value}")
            
except Exception as e:
    print(f"⚠️ Erreur évaluation: {e}")

# Test génération avec le modèle fine-tuné
print("\n🧪 Test de génération:")
print("-" * 30)

# Fonction de test du modèle
def test_surveillance_model(image, prompt, max_new_tokens=150):
    """
    Test du modèle fine-tuné sur une image de surveillance.
    """
    try:
        # Préparation du prompt au format LLaVA
        conversation = [
            {"role": "user", "content": f"<image>\n{prompt}"}
        ]
        
        # Formatage pour le processeur
        formatted_prompt = processor.apply_chat_template(
            conversation, 
            tokenize=False, 
            add_generation_prompt=True
        )
        
        # Préparation des inputs
        inputs = processor(
            text=formatted_prompt,
            images=image,
            return_tensors="pt"
        )
        
        # Génération
        with torch.no_grad():
            outputs = model.generate(
                **inputs,
                max_new_tokens=max_new_tokens,
                temperature=0.1,
                do_sample=True,
                pad_token_id=processor.tokenizer.eos_token_id,
            )
        
        # Décodage
        generated_text = processor.batch_decode(
            outputs, 
            skip_special_tokens=True
        )[0]
        
        # Extraction de la réponse
        if "ASSISTANT:" in generated_text:
            response = generated_text.split("ASSISTANT:")[-1].strip()
        else:
            response = generated_text.strip()
        
        return response
        
    except Exception as e:
        return f"Erreur de génération: {e}"

# Tests sur différents scénarios
test_scenarios = [
    {
        "name": "Analyse de sécurité générale",
        "prompt": "Analysez cette scène de surveillance et déterminez le niveau de suspicion. Utilisez vos outils d'analyse si nécessaire."
    },
    {
        "name": "Détection comportementale",
        "prompt": "Y a-t-il des comportements suspects dans cette image ? Quelles actions recommandez-vous ?"
    },
    {
        "name": "Orchestration d'outils",
        "prompt": "Analysez cette scène en utilisant les outils object_detector et behavior_analyzer. Fournissez un rapport détaillé."
    }
]

# Test sur les images du dataset de validation
if len(val_dataset) > 0:
    for i, scenario in enumerate(test_scenarios):
        if i < len(val_dataset):
            test_image = val_dataset[i]["image"]
            
            print(f"\n🔍 Test {i+1}: {scenario['name']}")
            print(f"❓ Question: {scenario['prompt']}")
            
            # Génération de la réponse
            response = test_surveillance_model(test_image, scenario["prompt"])
            
            print(f"🤖 Réponse: {response[:300]}..." if len(response) > 300 else f"🤖 Réponse: {response}")
            print("-" * 50)

print("\n✅ Tests de génération terminés")

In [ ]:
# Comparaison avec le modèle de base (si la mémoire le permet)
print("🆚 COMPARAISON MODÈLE BASE vs FINE-TUNÉ")
print("=" * 50)

try:
    # Chargement du modèle de base pour comparaison
    print("⏳ Chargement du modèle de base pour comparaison...")
    base_model = LlavaNextForConditionalGeneration.from_pretrained(
        MODEL_NAME,
        torch_dtype=torch.float16 if torch.cuda.is_available() else torch.float32,
        device_map="auto" if torch.cuda.is_available() else None,
        quantization_config=bnb_config  # Même quantization
    )
    
    print("✅ Modèle de base chargé")
    
    # Test comparatif sur une image
    if len(val_dataset) > 0:
        test_image = val_dataset[0]["image"]
        test_prompt = "Analysez cette scène de surveillance et déterminez s'il y a des comportements suspects."
        
        print(f"\n🖼️ Test comparatif sur image de validation")
        print(f"❓ Prompt: {test_prompt}")
        
        # Réponse du modèle de base
        print("\n📊 Modèle de BASE:")
        base_inputs = processor(
            text=f"<image>\nUSER: {test_prompt}\nASSISTANT:",
            images=test_image,
            return_tensors="pt"
        )
        
        with torch.no_grad():
            base_outputs = base_model.generate(
                **base_inputs,
                max_new_tokens=100,
                temperature=0.1,
                do_sample=True,
                pad_token_id=processor.tokenizer.eos_token_id,
            )
        
        base_response = processor.batch_decode(
            base_outputs, skip_special_tokens=True
        )[0].split("ASSISTANT:")[-1].strip()
        
        print(f"🤖 {base_response}")
        
        # Réponse du modèle fine-tuné
        print("\n🎯 Modèle FINE-TUNÉ:")
        finetuned_response = test_surveillance_model(test_image, test_prompt, max_new_tokens=100)
        print(f"🤖 {finetuned_response}")
        
        print("\n📝 Analyse comparative:")
        print("  • Le modèle fine-tuné devrait montrer:")
        print("    - Vocabulaire spécialisé surveillance")
        print("    - Mention d'outils spécifiques")
        print("    - Niveaux de suspicion structurés")
        print("    - Recommandations d'actions concrètes")
    
    # Nettoyage mémoire
    del base_model
    if torch.cuda.is_available():
        torch.cuda.empty_cache()
        
except Exception as e:
    print(f"⚠️ Comparaison impossible (mémoire insuffisante): {e}")
    print("💡 Le modèle fine-tuné est prêt à être utilisé individuellement")

print("\n✅ Évaluation terminée")

## 💾 Sauvegarde et Déploiement

In [ ]:
from huggingface_hub import HfApi, login
import json
from datetime import datetime
import os

print("💾 SAUVEGARDE ET DÉPLOIEMENT DU MODÈLE")
print("=" * 45)

# 1. Sauvegarde locale détaillée
print("🗂️ Sauvegarde locale...")

# Sauvegarde du modèle LoRA
if RECOMMENDED_CONFIG["use_lora"]:
    lora_output_dir = f"{OUTPUT_DIR}/lora_adapters"
    model.save_pretrained(lora_output_dir)
    print(f"  ✅ Adaptateurs LoRA: {lora_output_dir}")

# Sauvegarde du processeur
processor.save_pretrained(OUTPUT_DIR)
print(f"  ✅ Processeur: {OUTPUT_DIR}")

# Métadonnées du fine-tuning
metadata = {
    "model_name": MODEL_NAME,
    "fine_tuning_date": datetime.now().isoformat(),
    "training_config": {
        "epochs": num_epochs,
        "batch_size": batch_size,
        "learning_rate": training_args.learning_rate,
        "lora_rank": RECOMMENDED_CONFIG["lora_rank"] if RECOMMENDED_CONFIG["use_lora"] else None,
        "quantization": "4bit" if RECOMMENDED_CONFIG["load_in_4bit"] else None,
    },
    "dataset_info": {
        "train_samples": len(train_dataset),
        "val_samples": len(val_dataset),
        "scenarios": list(scenario_counts.keys()),
    },
    "performance": {
        "training_loss": training_result.training_loss if 'training_result' in locals() else None,
        "total_steps": training_result.global_step if 'training_result' in locals() else None,
    },
    "gpu_info": {
        "device": torch.cuda.get_device_name(0) if torch.cuda.is_available() else "CPU",
        "memory_gb": torch.cuda.get_device_properties(0).total_memory / 1e9 if torch.cuda.is_available() else None,
    }
}

with open(f"{OUTPUT_DIR}/training_metadata.json", "w") as f:
    json.dump(metadata, f, indent=2)
print(f"  ✅ Métadonnées: {OUTPUT_DIR}/training_metadata.json")

# 2. Script de chargement
loading_script = f'''
# Script de chargement du modèle fine-tuné
from transformers import LlavaNextForConditionalGeneration, LlavaNextProcessor
from peft import PeftModel
import torch

# Chargement du modèle de base
base_model = LlavaNextForConditionalGeneration.from_pretrained(
    "{MODEL_NAME}",
    torch_dtype=torch.float16,
    device_map="auto"
)

# Chargement des adaptateurs LoRA
model = PeftModel.from_pretrained(
    base_model, 
    "{lora_output_dir if RECOMMENDED_CONFIG['use_lora'] else OUTPUT_DIR}"
)

# Chargement du processeur
processor = LlavaNextProcessor.from_pretrained("{OUTPUT_DIR}")

print("✅ Modèle surveillance fine-tuné chargé !")
'''

with open(f"{OUTPUT_DIR}/load_model.py", "w") as f:
    f.write(loading_script)
print(f"  ✅ Script de chargement: {OUTPUT_DIR}/load_model.py")

# 3. Documentation du modèle
model_card = f'''
# 🕵️ Modèle VLM Fine-tuné pour Surveillance

## Description
Modèle Vision-Language basé sur LLaVA-NeXT fine-tuné spécialement pour l'analyse de surveillance en grande distribution.

## Capacités
- ✅ Analyse de scènes de surveillance
- ✅ Détection de comportements suspects
- ✅ Orchestration d'outils spécialisés
- ✅ Recommandations d'actions sécurisées

## Configuration d'Entraînement
- **Modèle de base**: {MODEL_NAME}
- **Méthode**: {"LoRA" if RECOMMENDED_CONFIG["use_lora"] else "Fine-tuning complet"}
- **Rang LoRA**: {RECOMMENDED_CONFIG["lora_rank"] if RECOMMENDED_CONFIG["use_lora"] else "N/A"}
- **Quantization**: {"4-bit" if RECOMMENDED_CONFIG["load_in_4bit"] else "Aucune"}
- **Époques**: {num_epochs}
- **Taille dataset**: {len(train_dataset)} (train) + {len(val_dataset)} (val)

## Utilisation
```python
from transformers import LlavaNextForConditionalGeneration, LlavaNextProcessor
from peft import PeftModel

# Chargement du modèle
base_model = LlavaNextForConditionalGeneration.from_pretrained(
    "{MODEL_NAME}",
    torch_dtype=torch.float16,
    device_map="auto"
)

model = PeftModel.from_pretrained(base_model, "./lora_adapters")
processor = LlavaNextProcessor.from_pretrained("./")

# Analyse d'une image
prompt = "Analysez cette scène de surveillance et déterminez le niveau de suspicion."
inputs = processor(text=f"<image>\\nUSER: {{prompt}}\\nASSISTANT:", images=image, return_tensors="pt")
outputs = model.generate(**inputs, max_new_tokens=150)
response = processor.batch_decode(outputs, skip_special_tokens=True)[0]
```

## Performance
{f"- **Loss finale**: {training_result.training_loss:.4f}" if 'training_result' in locals() else "- **Loss finale**: En cours d'évaluation"}
- **GPU utilisé**: {torch.cuda.get_device_name(0) if torch.cuda.is_available() else "CPU"}
{f"- **Durée d'entraînement**: {training_duration/60:.1f} minutes" if 'training_duration' in locals() else ""}

## Limitations
- Entraîné sur un dataset de démonstration (utiliser vos données réelles en production)
- Optimisé pour la surveillance en magasins (peut nécessiter adaptation pour autres contextes)
- Nécessite validation humaine pour déploiement critique

## Citation
```
@misc{{surveillance_vlm_2024,
  title={{Vision-Language Model for Intelligent Surveillance}},
  author={{Système de Surveillance Intelligente}},
  year={{2024}},
  note={{Fine-tuned from {MODEL_NAME}}}
}}
```
'''

with open(f"{OUTPUT_DIR}/README.md", "w") as f:
    f.write(model_card)
print(f"  ✅ Documentation: {OUTPUT_DIR}/README.md")

# 4. Archivage pour téléchargement
print("\n📦 Création de l'archive...")
import shutil

archive_name = f"surveillance_vlm_finetuned_{datetime.now().strftime('%Y%m%d_%H%M')}"
try:
    shutil.make_archive(archive_name, 'zip', OUTPUT_DIR)
    print(f"  ✅ Archive créée: {archive_name}.zip")
except Exception as e:
    print(f"  ⚠️ Erreur création archive: {e}")

# 5. Option de téléchargement sur Colab
if 'google.colab' in str(get_ipython()):
    try:
        from google.colab import files
        
        print("\n📥 Téléchargement des fichiers sur Colab:")
        
        # Télécharger l'archive principale
        if os.path.exists(f"{archive_name}.zip"):
            files.download(f"{archive_name}.zip")
            print(f"  ✅ Archive téléchargée")
        
        # Télécharger les métadonnées
        if os.path.exists(f"{OUTPUT_DIR}/training_metadata.json"):
            files.download(f"{OUTPUT_DIR}/training_metadata.json")
            print(f"  ✅ Métadonnées téléchargées")
            
    except Exception as e:
        print(f"  ⚠️ Téléchargement Colab non disponible: {e}")

print("\n✅ Sauvegarde terminée !")
print(f"📂 Dossier principal: {OUTPUT_DIR}")
print(f"📋 Fichiers sauvegardés:")
print(f"  • Modèle/Adaptateurs LoRA")
print(f"  • Processeur")
print(f"  • Métadonnées d'entraînement")
print(f"  • Script de chargement")
print(f"  • Documentation README")
print(f"  • Archive ZIP (si disponible)")

## 🔗 Intégration dans le Système de Surveillance

In [ ]:
# Exemple d'intégration du modèle fine-tuné dans le système principal

print("🔗 INTÉGRATION DANS LE SYSTÈME DE SURVEILLANCE")
print("=" * 50)

# Code d'exemple pour remplacer le VLM dans le système principal
integration_code = f'''
# Modification à apporter dans src/core/vlm/model.py
# pour utiliser le modèle fine-tuné

class VisionLanguageModel:
    def __init__(self, model_name: str = "custom", **kwargs):
        if model_name == "custom":
            # Chargement du modèle fine-tuné
            self.base_model = LlavaNextForConditionalGeneration.from_pretrained(
                "{MODEL_NAME}",
                torch_dtype=torch.float16,
                device_map="auto"
            )
            
            # Chargement des adaptateurs LoRA
            self.model = PeftModel.from_pretrained(
                self.base_model, 
                "{OUTPUT_DIR}/lora_adapters"
            )
            
            self.processor = LlavaNextProcessor.from_pretrained("{OUTPUT_DIR}")
        else:
            # Modèle standard
            self.model = LlavaNextForConditionalGeneration.from_pretrained(model_name)
            self.processor = LlavaNextProcessor.from_pretrained(model_name)
    
    async def analyze_frame(self, request: AnalysisRequest) -> AnalysisResponse:
        # La logique d'analyse reste la même
        # Le modèle fine-tuné donnera des réponses plus précises
        ...

# Usage dans le système principal:
vlm = VisionLanguageModel(model_name="custom")
'''

with open(f"{OUTPUT_DIR}/integration_guide.py", "w") as f:
    f.write(integration_code)

print("✅ Guide d'intégration créé")

# Instructions d'utilisation
print("\n📋 ÉTAPES D'INTÉGRATION:")
print("1. 📥 Télécharger les fichiers du modèle fine-tuné")
print("2. 📂 Placer les fichiers dans le dossier du projet")
print("3. 🔧 Modifier src/core/vlm/model.py selon integration_guide.py")
print("4. 🧪 Tester avec le système complet")
print("5. 📊 Évaluer les performances sur vos données réelles")

print("\n💡 RECOMMANDATIONS:")
print("• Utilisez des données réelles de votre environnement pour un meilleur fine-tuning")
print("• Ajustez les prompts selon vos besoins spécifiques")
print("• Évaluez régulièrement les performances en production")
print("• Considérez un fine-tuning itératif avec feedback utilisateur")

print("\n🎯 MÉTRIQUES À SURVEILLER EN PRODUCTION:")
print("• Précision des détections de comportements suspects")
print("• Taux de faux positifs vs faux négatifs")
print("• Temps de réponse du modèle")
print("• Cohérence des recommandations d'outils")
print("• Satisfaction des équipes de sécurité")

## 🎯 Conclusion et Prochaines Étapes

### ✅ Ce que vous avez accompli :

1. **🧠 Fine-tuning VLM** : Adaptation de LLaVA-NeXT pour la surveillance
2. **⚡ Optimisation LoRA** : Fine-tuning efficace avec ressources limitées
3. **🛠️ Tool-calling** : Capacités d'orchestration d'outils intégrées
4. **📊 Évaluation** : Tests qualitatifs et comparaisons
5. **💾 Déploiement** : Sauvegarde et guide d'intégration

### 📈 Performances Attendues :

Le modèle fine-tuné devrait montrer :
- **🎯 Vocabulaire spécialisé** surveillance et sécurité
- **🔧 Mentions d'outils** appropriés selon le contexte  
- **📊 Niveaux de suspicion** structurés (LOW/MEDIUM/HIGH/CRITICAL)
- **💡 Recommandations** d'actions concrètes
- **🤖 Tool-calling** naturel et contextuel

### 🚀 Prochaines Étapes Recommandées :

1. **📊 Dataset Réel** : Collectez et annotez vos propres données de surveillance
2. **🔄 Fine-tuning Itératif** : Améliorez progressivement avec feedback terrain
3. **📈 Évaluation Quantitative** : Métriques objectives (BLEU, ROUGE, métriques métier)
4. **🧪 Tests A/B** : Comparaison avec modèle de base en conditions réelles
5. **🔧 Optimisation** : Quantization, distillation pour déploiement edge

### 💡 Conseils pour la Production :

- **📝 Collecte Continue** : Enrichissez le dataset avec nouveaux scénarios
- **🔍 Monitoring** : Surveillez la qualité des réponses en temps réel
- **👥 Feedback Loop** : Intégrez retours des équipes de sécurité
- **🛡️ Validation Humaine** : Gardez une supervision pour cas critiques
- **📚 Documentation** : Maintenez guides d'utilisation à jour

---

**🎉 Félicitations ! Vous maîtrisez maintenant le fine-tuning de modèles VLM pour la surveillance.**

**📚 Ressources Utiles :**
- [Documentation Transformers](https://huggingface.co/transformers/)
- [Guide PEFT/LoRA](https://github.com/huggingface/peft)
- [LLaVA Paper](https://arxiv.org/abs/2304.08485)
- [Système de Surveillance Complet](https://github.com/elfried-kinzoun/intelligent-surveillance-system)

*Développé pour révolutionner la surveillance intelligente avec l'IA*